In [1]:
import numpy as np
import os
from skimage.io import imread, imsave
from scipy.ndimage import distance_transform_edt
import matplotlib.pyplot as plt

In [12]:
# Image frame to start from and step between frames
start_frame = 0
step = 1
nframes = 60
#pos = 0
# Position and velocity arrays from velocimetry

position = 2
path = '/media/c1046372/Expansion/Thesis GY/3. Analyzed files'
scope_name = 'Ti scope'
exp_date = '2023_11_28'
velocity_folder = 'velocity_data'
masks_folder = 'contour_masks'
results_folder = 'results'
dnas = {'pLPT20&pLPT41': 'pLPT20&41', 'pLPT119&pLPT41': 'pLPT119&41', 'pAAA': 'pAAA', 'pLPT107&pLPT41': 'pLPT107&41'}
scopes = {'Tweez scope': 'TiTweez', 'Ti scope': 'Ti'}
#vector = 'pLPT20&pLPT41'
vector = 'pAAA'
vel = np.load(os.path.join(path,scope_name,exp_date,velocity_folder,f'pos{position}','vel.np.npy'))
pos = np.load(os.path.join(path,scope_name,exp_date,velocity_folder,f'pos{position}','pos.np.npy'))

# Size of data
nx, ny, nt, _ = vel.shape

fname = f'{exp_date}_10x_1.0x_{dnas[vector]}_{scopes[scope_name]}_Pos{position}.ome.tif'
#im_all = imread(f'/media/c1046372/Expansion/Thesis GY/3. Analyzed files/Ti scope/2023_11_15/2023_11_15_10x_1.0x_{}_Ti_Pos0.ome.tif')
edt = np.load(os.path.join(path,scope_name,exp_date,results_folder,f'pos{position}','edt.npy'))
mask_all = imread(os.path.join(path,scope_name,exp_date,masks_folder,'mask_'+fname))
mask_all = mask_all > 0

_, edtnx, edtny = edt.shape
# mask_all = np.zeros(im_all.shape[:3])
x, y = np.meshgrid(np.arange(edtnx), np.arange(edtny))
# cx,cy = 320,500
# r = np.sqrt((x-cy)**2 + (y-cx)**2)
# mask = r<300
# for frame in range(im_all.shape[0]):
#    mask_all[frame,:,:] = mask
# mask_all = (mask_all==True)*1

# Make arrays to store results
radpos = np.zeros((nt, nx, ny))
vmag = np.zeros((nt, nx, ny))
vrad = np.zeros((nt, nx, ny))
vtheta = np.zeros((nt, nx, ny))

#edt = np.zeros((nt, 1200, 1200))


In [13]:
# Process the data and save results
for frame in range(nt):
    print(f'Processing frame {frame}')

    mask = mask_all[start_frame + frame * step + 1, :, :]
    cx = x[mask > 0].mean()
    cy = y[mask > 0].mean()

    vx = vel[:, :, frame, 0]
    vy = vel[:, :, frame, 1]

    # Subtract drift from velocities
    vx -= np.nanmean(vx)
    vy -= np.nanmean(vy)

    # Compute distance of each pixel from colony edge
    #edt[frame, :, :] = distance_transform_edt(mask)
    # edt[frame,:,:] = distance_transform_edt(mask_all[frame*step,:,:])

    # Get direction to colony edge as negative of gradient of distance
    gradx, grady = np.gradient(edt[start_frame + frame * step + 1, :, :])
    gradx[mask == 0] = np.nan
    grady[mask == 0] = np.nan
    px = pos[:, :, frame, 0].astype(int)
    py = pos[:, :, frame, 1].astype(int)
    pnorm = np.sqrt((px - cx) ** 2 + (py - cy) ** 2)

    gx = np.zeros((nx, ny))
    gy = np.zeros((nx, ny))
    for ix in range(nx):
        for iy in range(ny):
            gx[ix, iy] = -np.nanmean(gradx[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])
            gy[ix, iy] = -np.nanmean(grady[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])
            # radpos[frame,ix,iy] = np.nanmean(edt[frame, px[ix,iy]-32:px[ix,iy]+32, py[ix,iy]-32:py[ix,iy]+32])
    # Compute magnitude of velocities in radial direction
    velnorm = np.sqrt(vx ** 2 + vy ** 2)
    gnorm = np.sqrt(gx ** 2 + gy ** 2)
    vmag[frame, :, :] = vx * gx + vy * gy
    vrad[frame, :, :] = vmag[frame, :, :] / velnorm / gnorm
    vperp = vx * gy - vy * gx
    vtheta[frame, :, :] = vperp / velnorm / gnorm

    # Radial position of each grid square
    radpos[frame, :, :] = edt[frame, px + 16, py + 16]

# Area and estimated radius of colony
# area = mask_all[start_frame:start_frame + nt*step:step,:,:].sum(axis=(1,2))
#area = mask_all[start_frame:start_frame + nt * step:step, :, :].sum(axis=(1, 2))
#radius = np.sqrt(area / np.pi)

# Save results

path_save = os.path.join(path,scope_name,exp_date,results_folder,f'pos{position}')
np.save(os.path.join(path_save,'radpos.npy'), radpos)
#np.save(os.path.join(path_save,'radpos.npy')'results/edt.npy', edt)
np.save(os.path.join(path_save,'vmag.npy'), vmag)
np.save(os.path.join(path_save,'vrad.npy'), vrad)
np.save(os.path.join(path_save,'vtheta.npy'), vtheta)
#np.save(os.path.join(path_save,'radpos.npy')'results/area.npy', area)
#np.save(os.path.join(path_save,'radpos.npy')'results/radius.npy', radius)


Processing frame 0
Processing frame 1
Processing frame 2
Processing frame 3


/tmp/ipykernel_44724/2129167845.py:32: RuntimeWarning: Mean of empty slice
  gx[ix, iy] = -np.nanmean(gradx[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])
/tmp/ipykernel_44724/2129167845.py:33: RuntimeWarning: Mean of empty slice
  gy[ix, iy] = -np.nanmean(grady[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])


Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39
Processing frame 40
Processing frame 41
Processing frame 42
Processing frame 43
Processing frame 44
Processing frame 45
Processing frame 46
Processing frame 47
Processing frame 48
Processing frame 49
Processing frame 50
Processing frame 51
Processing frame 52
Processing frame 53
Proces

In [15]:
edt.shape

(216, 1200, 1200)

In [16]:
radpos.shape

(59, 37, 37)

In [17]:
vel.shape

(37, 37, 59, 2)

In [18]:
pos.shape

(37, 37, 59, 2)

In [19]:
# Image frame to start from and step between frames
start_frame = 0
step = 1
nframes = 60
#pos = 0
# Position and velocity arrays from velocimetry

position = 0
path = '/media/c1046372/Expansion/Thesis GY/3. Analyzed files'
scope_name = 'Ti scope'
exp_date = '2023_11_15'
velocity_folder = 'velocity_data'
masks_folder = 'contour_masks'
results_folder = 'results'
dnas = {'pLPT20&pLPT41': 'pLPT20&41', 'pLPT119&pLPT41': 'pLPT119&41', 'pAAA': 'pAAA', 'pLPT107&pLPT41': 'pLPT107&41'}
scopes = {'Tweez scope': 'TiTweez', 'Ti scope': 'Ti'}
vector = 'pLPT20&pLPT41'
#vector = 'pAAA'
vel = np.load(os.path.join(path,scope_name,exp_date,velocity_folder,f'pos{position}','vel.np.npy'))
pos = np.load(os.path.join(path,scope_name,exp_date,velocity_folder,f'pos{position}','pos.np.npy'))

# Size of data
nx, ny, nt, _ = vel.shape

fname = f'{exp_date}_10x_1.0x_{dnas[vector]}_{scopes[scope_name]}_Pos{position}.ome.tif'
#im_all = imread(f'/media/c1046372/Expansion/Thesis GY/3. Analyzed files/Ti scope/2023_11_15/2023_11_15_10x_1.0x_{}_Ti_Pos0.ome.tif')
edt = np.load(os.path.join(path,scope_name,exp_date,results_folder,f'pos{position}','edt.npy'))
mask_all = imread(os.path.join(path,scope_name,exp_date,masks_folder,'mask_'+fname))
mask_all = mask_all > 0

_, edtnx, edtny = edt.shape
# mask_all = np.zeros(im_all.shape[:3])
x, y = np.meshgrid(np.arange(edtnx), np.arange(edtny))
# cx,cy = 320,500
# r = np.sqrt((x-cy)**2 + (y-cx)**2)
# mask = r<300
# for frame in range(im_all.shape[0]):
#    mask_all[frame,:,:] = mask
# mask_all = (mask_all==True)*1

# Make arrays to store results
radpos = np.zeros((nt, nx, ny))
vmag = np.zeros((nt, nx, ny))
vrad = np.zeros((nt, nx, ny))
vtheta = np.zeros((nt, nx, ny))

#edt = np.zeros((nt, 1200, 1200))

In [20]:
# Process the data and save results
for frame in range(nt):
    print(f'Processing frame {frame}')

    mask = mask_all[start_frame + frame * step + 1, :, :]
    cx = x[mask > 0].mean()
    cy = y[mask > 0].mean()

    vx = vel[:, :, frame, 0]
    vy = vel[:, :, frame, 1]

    # Subtract drift from velocities
    vx -= np.nanmean(vx)
    vy -= np.nanmean(vy)

    # Compute distance of each pixel from colony edge
    #edt[frame, :, :] = distance_transform_edt(mask)
    # edt[frame,:,:] = distance_transform_edt(mask_all[frame*step,:,:])

    # Get direction to colony edge as negative of gradient of distance
    gradx, grady = np.gradient(edt[start_frame + frame * step + 1, :, :])
    gradx[mask == 0] = np.nan
    grady[mask == 0] = np.nan
    px = pos[:, :, frame, 0].astype(int)
    py = pos[:, :, frame, 1].astype(int)
    pnorm = np.sqrt((px - cx) ** 2 + (py - cy) ** 2)

    gx = np.zeros((nx, ny))
    gy = np.zeros((nx, ny))
    for ix in range(nx):
        for iy in range(ny):
            gx[ix, iy] = -np.nanmean(gradx[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])
            gy[ix, iy] = -np.nanmean(grady[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])
            # radpos[frame,ix,iy] = np.nanmean(edt[frame, px[ix,iy]-32:px[ix,iy]+32, py[ix,iy]-32:py[ix,iy]+32])
    # Compute magnitude of velocities in radial direction
    velnorm = np.sqrt(vx ** 2 + vy ** 2)
    gnorm = np.sqrt(gx ** 2 + gy ** 2)
    vmag[frame, :, :] = vx * gx + vy * gy
    vrad[frame, :, :] = vmag[frame, :, :] / velnorm / gnorm
    vperp = vx * gy - vy * gx
    vtheta[frame, :, :] = vperp / velnorm / gnorm

    # Radial position of each grid square
    radpos[frame, :, :] = edt[frame, px + 16, py + 16]

# Area and estimated radius of colony
# area = mask_all[start_frame:start_frame + nt*step:step,:,:].sum(axis=(1,2))
#area = mask_all[start_frame:start_frame + nt * step:step, :, :].sum(axis=(1, 2))
#radius = np.sqrt(area / np.pi)

# Save results

path_save = os.path.join(path,scope_name,exp_date,results_folder,f'pos{position}')
np.save(os.path.join(path_save,'radpos.npy'), radpos)
#np.save(os.path.join(path_save,'radpos.npy')'results/edt.npy', edt)
np.save(os.path.join(path_save,'vmag.npy'), vmag)
np.save(os.path.join(path_save,'vrad.npy'), vrad)
np.save(os.path.join(path_save,'vtheta.npy'), vtheta)
#np.save(os.path.join(path_save,'radpos.npy')'results/area.npy', area)
#np.save(os.path.join(path_save,'radpos.npy')'results/radius.npy', radius)


Processing frame 0


/tmp/ipykernel_44724/2129167845.py:32: RuntimeWarning: Mean of empty slice
  gx[ix, iy] = -np.nanmean(gradx[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])
/tmp/ipykernel_44724/2129167845.py:33: RuntimeWarning: Mean of empty slice
  gy[ix, iy] = -np.nanmean(grady[px[ix, iy]:px[ix, iy] + 64, py[ix, iy]:py[ix, iy] + 64])


IndexError: index 1200 is out of bounds for axis 1 with size 1200

In [21]:
edt.shape

(216, 1200, 1200)

In [22]:
radpos.shape

(59, 75, 75)

In [23]:
vel.shape

(75, 75, 59, 2)

In [24]:
pos.shape

(75, 75, 59, 2)

In [25]:
frame

0

In [28]:
radpos[frame, :, :] = edt[frame, px + 16, py + 16]

IndexError: index 1200 is out of bounds for axis 1 with size 1200